In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
load_dotenv()


/var/folders/8j/ybqbv42143jfglvsf83t5s_c0000gn/T/ipykernel_61878/3011661058.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


True

In [2]:
# Get the API key from the .env file
API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY

In [3]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.query_engine import PandasQueryEngine

In [25]:
df = pd.read_csv('data_llm/work-nature.csv')
# add a column for permit approval time
# convert issue date to datetime
data_df = df
data_df['issue_date'] = pd.to_datetime(data_df['issue_date'])
data_df['start_date'] = pd.to_datetime(data_df['start_date'])

# get approval time in days from start date and issue date
data_df['approval_time'] = data_df['issue_date'] - data_df['start_date']

data_df.dropna(inplace=True)
data_df.isnull().sum()

data_df.head()

,start_date,issue_date,estimated_work_cost,issued_permit_cost,building_type_description,building_category_description,work_nature,approval_time
0,1994-05-16,1994-05-18,110000.0,380.0,Résidentiel,Permis ancien système,"EXPIRATION DU PERMIS 18 NOV. 1994., CONSTRUCTI...",2 days
1,1997-08-06,1997-08-07,8500.0,34.0,Résidentiel,Permis ancien système,"RENOVATION INTERIEURE D'UNE RESI-, DENCE., CON...",1 days
2,1997-08-08,1997-09-24,1000.0,40.0,Résidentiel,Permis ancien système,"DEMOLITION D'UNE RESIDENCE DE 9.44m, X 14.02 m...",47 days
3,1997-08-08,1997-09-24,1000.0,40.0,Commercial,Permis ancien système,"DEMOLITION D'UNE GRANGE DE 9.44 m X, 11.88 m, ...",47 days
4,1997-08-08,1997-09-24,150000.0,450.0,Résidentiel,Permis ancien système,"CONSTRUCTION D'UNE RESIDENCE UNIFA-, MILIALE D...",47 days


In [26]:
# drop the columns that are not needed
data_df = data_df.drop(['issue_date', 'start_date'], axis=1)


In [28]:
# save the data to a csv file
data_df.to_csv('data_llm/work-nature-approval-time.csv', index=False)

In [ ]:
## Trying out PandasQueryEngine
df = pd.read_csv('data_llm/work-nature.csv')
query_engine = PandasQueryEngine(df=df, verbose=True)
response = query_engine.query(
    "Please show the first 10 rows. \
    Convert the response From French to English.?"
)
print(response)

In [29]:
## Trying out SimpleCSVReader

from pathlib import Path
from llama_index import download_loader
SimpleCSVReader = download_loader("SimpleCSVReader")
loader = SimpleCSVReader()



In [30]:
## with the new dataset with approval times
documents = loader.load_data(file=Path('data_llm/work-nature-approval-time.csv'))

In [31]:
from llama_index import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 900/900 [00:19<00:00, 47.22it/s]


In [35]:
query_engine=index.as_query_engine()

response = query_engine.query("what are the average approval times in days for the three types of work nature? \
Show proof of why you think this is correct.\
Provide the response only in English.")

print(response)

The average approval times in days for the three types of work nature are as follows:

1. Construction: The average approval time for construction work is approximately 23 days. This is based on the approval times provided for various construction projects in the given context.

2. Renovation: The average approval time for renovation work is approximately 5 days. This is based on the approval times provided for various renovation projects in the given context.

3. Demolition: The average approval time for demolition work is approximately 47 days. This is based on the approval times provided for various demolition projects in the given context.

Please note that these average approval times are approximate and may vary depending on the specific circumstances of each project.
